# Inversion Experiments

In [ ]:
# For when connected to google colab
!git clone https://github.com/AdrSkapars/inversion_optimisation.git 
%cd inversion_optimisation
!uv pip install -e .

## Set Up Model

In [1]:
import sys
sys.path.insert(0, "/content/inversion_optimisation/src")

import torch
from datasets import load_dataset
from transformer_lens import HookedTransformer
import numpy as np
import random
import pickle
from tqdm import tqdm
import json
import logging
import time

from inversion_optimisation.utils import (
    get_paper_summary_stats_new, 
    DotDict,
    DATA_PATH,
)

/workspace/inversion_optimisation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the LLM model

device = ["cuda", "cpu"][0]
model_name = {
    # Simple models
    0: "attn-only-1l",
    1: "gelu-1l",
    2: "tiny-stories-1L-21M",
    
    ## Small models
    3: "tiny-stories-1M",
    4: "tiny-stories-3M",
    5: "tiny-stories-8M",
    6: "tiny-stories-28M",
    7: "tiny-stories-33M",
    8: "tiny-stories-instruct-33M",
    
    ## Large models
    9: "gpt2-small",
    10: "gpt2-medium",
    11: "gpt2-xl",
    12: "llama-7b",
    13: "meta-llama/Llama-3.2-1B",
    14: "meta-llama/Llama-3.2-1B-Instruct",
    15: "Qwen/Qwen2.5-0.5B",
    16: "Qwen/Qwen2.5-1.5B",
    17: "Qwen/Qwen2.5-3B",
    18: "pythia-1.4b",
}[7]

model = HookedTransformer.from_pretrained(model_name)#, device=device)
model = model.eval()

`torch_dtype` is deprecated! Use `dtype` instead!


Loaded pretrained model tiny-stories-33M into HookedTransformer


# ------------------------

## SODA Text-Only Inversion

In [4]:
from inversion_optimisation.algorithms.soda import onehot_text_search
from inversion_optimisation.inv_configs import SODA_TEXT_CFG

cfg = SODA_TEXT_CFG
# cfg = DotDict({
#     "learn_rate" : 0.065,
#     "decay_rate" : 0.9,
#     "betas" : (0.9,0.995),
#     "temp" : 0.05,
#     "reset_epoch" : 50,
#     "reinit_epoch" : 1500,
#     "reg_weight" : None,#9e-3,
#     "bias_correction" : False,
#     "target_sample" : {
#         0: "random",
#         1: "greedy"
#     }[1],
#     # "target_strategy" : {
#     #     0: "random",
#     #     1: "tinystories",
#     #     2: "reddit",
#     #     3: "wikipedia"
#     # }[0],
#     "init_strategy" : {
#         0: "loaded",
#         1: "normal",
#         2: "zeros",
#     }[2],
#     "save_folder": "OneHot_TinyStories33M_text",
#     "model_name": "tiny-stories-33M",
# })

In [ ]:
# Set parameters for dataset size
cfg.input_len = 1
cfg.output_len = 25
cfg.num_targets = 1000
cfg.max_epochs = 100 * 1000
cfg.max_batch_size = {
    "tiny-stories-33M" : [300,300,280,270,260,250,250,240,240,230],
}[cfg.model_name][cfg.input_len-1]
cfg.max_batch_size = cfg.max_batch_size - 15

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

batch_size = 1000
for batch in range(0, cfg.num_targets, batch_size):
    input_tokens = true_tokens[batch:batch+batch_size].to(device)
    output_tokens = model.generate(
        input_tokens,
        # min_new_tokens=cfg.output_len,
        max_new_tokens=cfg.output_len,
        # do_sample=True,
        do_sample=False,
        stop_at_eos=False,
        verbose=False,
        return_type="tokens",)[:,cfg.input_len:]
    if batch == 0:
        all_output_tokens = output_tokens
    else:
        all_output_tokens = torch.cat((all_output_tokens, output_tokens), dim=0)
with open(DATA_PATH / f"true_tokens_{cfg.num_targets}_{cfg.input_len}_{cfg.output_len}_greedy.pkl", 'wb') as file:
    pickle.dump(all_output_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# # Generate the initialisations used for all experiments
# tokens_list = []
# for _ in tqdm(range(cfg.num_targets)):
#     tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
#     tokens_list.append(tokens)
# initial_tokens = torch.cat(tokens_list, dim=0).to(device)
# with open(DATA_PATH / f'initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
#     pickle.dump(initial_tokens, file)

results, elapsed_time = onehot_text_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)


## SODA Logits Inversion

In [5]:
from inversion_optimisation.algorithms.soda import onehot_search
from inversion_optimisation.inv_configs import SODA_LOGITS_CFG, SODA_LOGITS_CFG_DIFF_MODELS

cfg = SODA_LOGITS_CFG
# cfg = SODA_LOGITS_CFG_DIFF_MODELS[cfg.model_name]
# cfg = DotDict({
#     "learn_rate" : 0.065,
#     "decay_rate" : 0.9,
#     "betas" : (0.9,0.995),
#     "temp" : 0.05,
#     "reset_epoch" : 50,
#     "reinit_epoch" : 1500,
#     "reg_weight" : None,#9e-3,
#     "bias_correction" : False,
#     # "target_strategy" : {
#     #     0: "random",
#     #     1: "tinystories",
#     #     2: "reddit",
#     #     3: "wikipedia",
#     #     4: "privacy",
#     # }[0],
#     "init_strategy" : {
#         0: "loaded",
#         1: "normal",
#         2: "zeros",
#     }[2],
#     "save_folder": "OneHot_TinyStories33M",
#     "model_name": "tiny-stories-33M",
# })

In [7]:
# Set parameters for dataset size
cfg.input_len = 1
cfg.num_targets = 1000
cfg.max_epochs = 100# * 1000
cfg.max_batch_size = {
    "tiny-stories-33M" : [1000,1000,1000,1000,1000,1000,1000,900,750,650,None, None, None, None, 500, 500, 475, 450, 410, 380, 375, 350, 325, 300],
    "gpt2-small": [1000,1000,1000,1000,1000,950,850,700,650,600],
    "Qwen/Qwen2.5-0.5B": [1000,700,500,400,300,240,220,200,180,100],
}[cfg.model_name][cfg.input_len-1]

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# # Generate the initialisations used for all experiments
# tokens_list = []
# for _ in tqdm(range(cfg.num_targets)):
#     tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
#     tokens_list.append(tokens)
# initial_tokens = torch.cat(tokens_list, dim=0).to(device)
# with open(DATA_PATH / f'initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
#     pickle.dump(initial_tokens, file)

results, elapsed_time = onehot_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:12<00:00, 77.69it/s]


TypeError: onehot_search() missing 1 required positional argument: 'model'

## GCG Text-Only Inversion

In [ ]:
from inversion_optimisation.algorithms.gcg import gcg_text_search
from inversion_optimisation.inv_configs import GCG_TEXT_CFG

cfg = GCG_TEXT_CFG
# cfg = DotDict({
#     "top_k" : 128,
#     "pos_choice" : {
#         0: "uniform",
#         1: "weighted",
#         2: "greedy",
#     }[0],
#     "token_choice" : {
#         0: "uniform",
#         1: "weighted",
#     }[0],
#     "num_mutations" : 1,
#     "target_sample" : {
#         0: "random",
#         1: "greedy"
#     }[1],
#     # "target_strategy" : {
#     #     0: "random",
#     #     1: "tinystories",
#     #     2: "reddit",
#     #     3: "wikipedia"
#     # }[0],
#     "init_strategy" : {
#         0: "loaded",
#         1: "zeros",
#     }[0],
#     "save_folder": "GCG_TinyStories33M_text",
#     "model_name": "tiny-stories-33M",
# })

In [ ]:
# Set parameters for dataset size
cfg.output_len = 25
cfg.input_len = 1
cfg.num_targets = 1000
cfg.num_candidates = (32*22)
cfg.adjusted_max_epochs = (32*22)
cfg.max_batch_size = {
    "tiny-stories-33M" : [300,300,280,270,260,250,250,240,240,230],
}[cfg.model_name][cfg.input_len-1]

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets and initialisations used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

batch_size = 1000
for batch in range(0, cfg.num_targets, batch_size):
    input_tokens = true_tokens[batch:batch+batch_size].to(device)
    output_tokens = model.generate(
        input_tokens,
        max_new_tokens=cfg.output_len,
        do_sample=False,
        stop_at_eos=False,
        verbose=False,
        return_type="tokens",)[:,cfg.input_len:]
    if batch == 0:
        all_output_tokens = output_tokens
    else:
        all_output_tokens = torch.cat((all_output_tokens, output_tokens), dim=0)
with open(DATA_PATH / f"true_tokens_{cfg.num_targets}_{cfg.input_len}_{cfg.output_len}_greedy.pkl", 'wb') as file:
    pickle.dump(all_output_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f"initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl", 'wb') as file:
    pickle.dump(true_tokens, file)

results, elapsed_time = gcg_text_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.adjusted_max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.adjusted_max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.adjusted_max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)

## GCG Logits Inversion

In [2]:
from inversion_optimisation.algorithms.gcg import gcg_search
from inversion_optimisation.inv_configs import GCG_LOGITS_CFG

cfg = GCG_LOGITS_CFG
cfg = DotDict({
    "top_k" : 128,
    "pos_choice" : {
        0: "uniform",
        1: "weighted",
        2: "greedy",
    }[0],
    "token_choice" : {
        0: "uniform",
        1: "weighted",
    }[0],
    "num_mutations" : 1,
    # "target_strategy" : {
    #     0: "random",
    #     1: "tinystories",
    #     2: "reddit",
    #     3: "wikipedia",
    #     4: "privacy",
    # }[0],
    "init_strategy" : {
        0: "loaded",
        1: "zeros",
    }[0],
    "save_folder": "GCG_TinyStories33M",
    "model_name": "tiny-stories-33M",
})

ModuleNotFoundError: No module named 'inversion_optimisation'

In [ ]:
# Set parameters for dataset size
cfg.input_len = 15
cfg.num_targets = 1000
cfg.num_candidates = (32*22)
cfg.adjusted_max_epochs = (32*22)
cfg.max_batch_size = {
    "tiny-stories-33M" : [1000,1000,1000,1000,1000,1000,1000,900,750,650,None, None, None, None, 500, 500, 475, 450, 420, 400, 375, 350, 325, 300],
}[cfg.model_name][cfg.input_len-1]

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets and initialisations used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f"initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl", 'wb') as file:
    pickle.dump(true_tokens, file)

results, elapsed_time = gcg_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.adjusted_max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.adjusted_max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.adjusted_max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)

## Inv. Model Text-Only Inversion

In [ ]:
from inversion_optimisation.algorithms.text_inv_model import (
    LLMInversionModel, 
    train_inversion_model, 
    test_inversion_model_dataset, 
    test_inversion_model
)
from inversion_optimisation.inv_configs import INV_MODEL_TEXT_CFG

cfg = INV_MODEL_TEXT_CFG
# cfg = DotDict({
#     "t5_model_name" : "t5-small",
#     "t5_tokenizer_name" : "t5-small",
#     "llm_model_name" : "roneneldan/TinyStories-33M",
#     "num_generation_tokens" : 25,
#     "seed" : 24,
#     "dataset_size" : 400000,
#     "min_seq_length" : 1,
#     "max_seq_length" : 10,
#     "max_length" : 16,
#     "val_split" : 0.1,
#     # "output_dir" : "/content/TextInvModel_Saves",
#     "batch_size" : 160,
#     "mini_batch_size" : 160,
#     "num_epochs" : 30,
#     "save_steps" : 3000000,
#     "warmup_steps" : 1000,
#     "num_workers" : 12,
#     "learning_rate" : 1e-3,
#     "weight_decay" : 0.05,
#     # "dataset" : ["random", "reddit", "tinystories"][0],
# })

In [ ]:
# Load initial model
inv_model = LLMInversionModel(
    t5_model_name=cfg.t5_model_name,
    t5_tokenizer_name=cfg.t5_tokenizer_name,
    llm_model_name=cfg.llm_model_name,
    num_generation_tokens=cfg.num_generation_tokens,
).to(device)

In [ ]:
# Set level of logging
# logging.getLogger().setLevel(logging.INFO)
# logging.getLogger().setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
# Train model
elapsed_time = time.time()
inv_model = train_inversion_model(
    model=inv_model,
    seed=cfg.seed, # Set random seed
    dataset_size=cfg.dataset_size,
    min_seq_length=cfg.min_seq_length,
    max_seq_length=cfg.max_seq_length,
    max_length=cfg.max_length,
    val_split=cfg.val_split,
    output_dir="data/TextInvModel_Saves",
    batch_size=cfg.batch_size,
    mini_batch_size=cfg.mini_batch_size,
    num_epochs=cfg.num_epochs,
    save_steps=cfg.save_steps,
    warmup_steps=cfg.warmup_steps,
    num_workers=cfg.num_workers,
    learning_rate=cfg.learning_rate,
    weight_decay=cfg.weight_decay,
    dataset=["random", "reddit", "tinystories"][0],
)
elapsed_time = time.time() - elapsed_time
print(f"Elapsed time: {elapsed_time:.2f}")

In [ ]:
# Evaluate model
num_targets = 1000
eval_batch_size = 200
input_len = 1
max_length = 16

# Load best model
inv_model = LLMInversionModel.from_pretrained("data/TextInvModel_Saves/best_model").to(device)

# Generate dataset
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Choose random (Random) dataset
tokens_list = []
for _ in tqdm(range(num_targets)):
    tokens = torch.randint(0, len(inv_model.tokenizer.vocab), (1, input_len)).to(device)
    tokens_list.append(tokens)
loaded_true_tokens = torch.cat(tokens_list, dim=0).to(device)
loaded_text_samples = inv_model.llm_tokenizer.batch_decode(loaded_true_tokens)

# # Choose reddit (NL OOD) dataset
# loaded_true_tokens = load_dataset_tokens("reddit", input_len, num_targets, include_bos=False, random_sentence=True, random_start=False)
# loaded_text_samples = inv_model.llm_tokenizer.batch_decode(loaded_true_tokens)

# # Choose tinystories (NL ID) dataset
# loaded_true_tokens = load_dataset_tokens("tinystories", input_len, num_targets, include_bos=False, random_sentence=True, random_start=False)
# loaded_text_samples = inv_model.llm_tokenizer.batch_decode(loaded_true_tokens)

# Run model
results, stats = test_inversion_model_dataset(inv_model, loaded_text_samples, eval_batch_size, input_len, max_length)

# with open(DATA_PATH / f'TextInvModel_Saves/stats_{input_len}_{num_targets}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'TextInvModel_Saves/results_{input_len}_{num_targets}.pkl', 'wb') as f:
#     pickle.dump(results, f)

In [ ]:
# Example test model
test_texts = ["Hey","Hello","I","So", 'American']
results = test_inversion_model(model, test_texts, max_length=8)

## Inv. Model Logits Inversion

In [ ]:
from inversion_optimisation.algorithms.inv_model import (
    LLMInversionModel, 
    train_inversion_model, 
    test_inversion_model_dataset, 
    test_inversion_model
)
from inversion_optimisation.inv_configs import INV_MODEL_LOGITS_CFG

cfg = INV_MODEL_LOGITS_CFG
# cfg = DotDict({
#     "t5_model_name" : "t5-small",
#     "t5_tokenizer_name" : "t5-small",
#     "llm_model_name" : "roneneldan/TinyStories-33M",
#     "unigram_beta" : 0.01,
#     "num_tokens" : 64,
#     "bottleneck_dim" : 32768,
#     "seed" : 24, # Set random seed
#     "dataset_size" : 400000,
#     "min_seq_length" : 1,
#     "max_seq_length" : 10,
#     "max_length" : 16,
#     "val_split" : 0.1,
#     # "output_dir" : "/content/InvModel_Saves",
#     "batch_size" : 80,
#     "mini_batch_size" : 80,
#     "num_epochs" : 30,
#     "save_steps" : 3000000,
#     "warmup_steps" : 1000,
#     "num_workers" : 12,
#     "learning_rate" : 2e-4,
#     "weight_decay" : 0.025,
# })

In [ ]:
# Load initial model
inv_model = LLMInversionModel(
    t5_model_name=cfg.t5_model_name,
    t5_tokenizer_name=cfg.t5_tokenizer_name,
    llm_model_name=cfg.llm_model_name,
    unigram_beta=cfg.unigram_beta,
    num_tokens=cfg.num_tokens,
    bottleneck_dim=cfg.bottleneck_dim,
).to(device)

In [ ]:
# Set level of logging
# logging.getLogger().setLevel(logging.INFO)
# logging.getLogger().setLevel(logging.WARNING)
logging.getLogger().setLevel(logging.ERROR)

In [ ]:
# Train model
elapsed_time = time.time()
inv_model = train_inversion_model(
    model=inv_model,
    seed=cfg.seed, # Set random seed
    dataset_size=cfg.dataset_size,
    min_seq_length=cfg.min_seq_length,
    max_seq_length=cfg.max_seq_length,
    max_length=cfg.max_length,
    val_split=cfg.val_split,
    output_dir="data/InvModel_Saves",
    batch_size=cfg.batch_size,
    mini_batch_size=cfg.mini_batch_size,
    num_epochs=cfg.num_epochs,
    save_steps=cfg.save_steps,
    warmup_steps=cfg.warmup_steps,
    num_workers=cfg.num_workers,
    learning_rate=cfg.learning_rate,
    weight_decay=cfg.weight_decay,
)
elapsed_time = time.time() - elapsed_time
print(f"Elapsed time: {elapsed_time:.2f}")

In [ ]:
# Evaluate model
num_targets = 1000
eval_batch_size = 200
input_len = 1
max_length = 32

# Load best model
inv_model = LLMInversionModel.from_pretrained("data/InvModel_Saves/best_model").to(device)

# Generate dataset
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

tokens_list = []
for _ in tqdm(range(num_targets)):
    tokens = torch.randint(0, len(inv_model.tokenizer.vocab), (1, input_len)).to(device)
    tokens_list.append(tokens)
loaded_true_tokens = torch.cat(tokens_list, dim=0).to(device)
loaded_text_samples = inv_model.llm_tokenizer.batch_decode(loaded_true_tokens)

# Run model
results, stats = test_inversion_model_dataset(inv_model, loaded_text_samples, eval_batch_size, input_len, max_length)

# with open(DATA_PATH / f'InvModel_Saves/stats_{input_len}_{num_targets}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'InvModel_Saves/results_{input_len}_{num_targets}.pkl', 'wb') as f:
#     pickle.dump(results, f)

In [ ]:
# Example test model
test_texts = ["Hey","Hello","I","So", 'American']
results = test_inversion_model(model, test_texts, max_length=8)

# ------------------------

## Embed Search Text-Only Inversion

In [ ]:
from inversion_optimisation.algorithms.embed_search import embed_text_search

cfg = DotDict({
    "learn_rate" : 0.0003,
    # "decay_rate" : 1,
    "betas" : (0.9,0.999),
    # "reset_epoch" : 999999,
    # "reinit_epoch" : 999999,
    # "reg_weight" : None,#9e-3,
    # "bias_correction" : True,
    "con_distance" : {
        0: "cos sim",
        1: "euc dist",
    }[0],
    "target_sample" : {
        0: "random",
        1: "greedy"
    }[1],
    "target_strategy" : {
        0: "random",
        1: "tinystories",
        2: "reddit",
        3: "wikipedia"
    }[0],
    "init_strategy" : {
        0: "loaded",
        1: "normal",
        2: "zeros",
    }[2],
    "save_folder": "Embed_TinyStories33M_text",
    "model_name": "tiny-stories-33M",
})

In [ ]:
# Set parameters for dataset size
cfg.input_len = 1
cfg.output_len = 25
cfg.num_targets = 1000
cfg.max_chunk_size = 20
cfg.max_epochs = 100 * 1000
cfg.max_batch_size = {
    "tiny-stories-33M" : [300,300,280,270,260,250,250,240,240,230],
}[cfg.model_name][cfg.input_len-1]
cfg.max_batch_size = cfg.max_batch_size - 15
cfg.check_con_epochs = 200

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

batch_size = 1000
for batch in range(0, cfg.num_targets, batch_size):
    input_tokens = true_tokens[batch:batch+batch_size].to(device)
    output_tokens = model.generate(
        input_tokens,
        # min_new_tokens=cfg.output_len,
        max_new_tokens=cfg.output_len,
        # do_sample=True,
        do_sample=False,
        stop_at_eos=False,
        verbose=False,
        return_type="tokens",)[:,cfg.input_len:]
    if batch == 0:
        all_output_tokens = output_tokens
    else:
        all_output_tokens = torch.cat((all_output_tokens, output_tokens), dim=0)
with open(DATA_PATH / f"true_tokens_{cfg.num_targets}_{cfg.input_len}_{cfg.output_len}_greedy.pkl", 'wb') as file:
    pickle.dump(all_output_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# # Generate the initialisations used for all experiments
# tokens_list = []
# for _ in tqdm(range(cfg.num_targets)):
#     tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
#     tokens_list.append(tokens)
# initial_tokens = torch.cat(tokens_list, dim=0).to(device)
# with open(DATA_PATH / f'initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
#     pickle.dump(initial_tokens, file)

results, elapsed_time = embed_text_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)


## Embed Search Logits Inversion

In [ ]:
from inversion_optimisation.algorithms.embed_search import embed_search

cfg = DotDict({
    "learn_rate" : 0.0003,
    # "decay_rate" : 1,
    "betas" : (0.9,0.999),
    # "reset_epoch" : 999999,
    # "reinit_epoch" : 999999,
    # "bias_correction" : True,
    "con_distance" : {
        0: "cos sim",
        1: "euc dist",
    }[0],
    "target_strategy" : {
        0: "random",
        1: "tinystories",
        2: "reddit",
        3: "wikipedia",
        4: "privacy",
    }[0],
    "init_strategy" : {
        0: "loaded",
        1: "normal",
        2: "zeros",
    }[2],
    "save_folder": "Embed_TinyStories33M",
    "model_name": "tiny-stories-33M",
})

In [ ]:
# Set parameters for dataset size
cfg.input_len = 1
cfg.num_targets = 1000
cfg.max_epochs = 100 * 1000
# cfg.max_chunk_size = [30,20,15,10,8,6,6,5,5,4][cfg.input_len-1] # L4 batch with 100 batch
cfg.max_chunk_size = [20,15,10,7,6,5,4,3,2,2][cfg.input_len-1] # L4 batch with 1000 batch
cfg.max_batch_size = [1000,1000,1000,1000,1000,1000,1000,1000,1000,1000][cfg.input_len-1] # L4 batch
cfg.check_con_epochs = 200

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# # Generate the initialisations used for all experiments
# tokens_list = []
# for _ in tqdm(range(cfg.num_targets)):
#     tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
#     tokens_list.append(tokens)
# initial_tokens = torch.cat(tokens_list, dim=0).to(device)
# with open(DATA_PATH / f'initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
#     pickle.dump(initial_tokens, file)

results, elapsed_time = embed_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)

## PEZ Text-Only Inversion

In [ ]:
from inversion_optimisation.algorithms.pez import pez_text_search
from inversion_optimisation.inv_configs import PEZ_TEXT_CFG

cfg = PEZ_TEXT_CFG
# cfg = DotDict({
#     "learn_rate" : 2.5,
#     "con_distance" : {
#         0: "cos sim",
#         1: "euc dist",
#     }[1],
#     "target_sample" : {
#         0: "random",
#         1: "greedy"
#     }[1],
#     "target_strategy" : {
#         0: "random",
#         1: "tinystories",
#         2: "reddit",
#         3: "wikipedia"
#     }[0],
#     "init_strategy" : {
#         0: "loaded",
#         1: "normal",
#         2: "zeros",
#     }[0],
#     "save_folder": "PEZ_TinyStories33M_text",
#     "model_name": "tiny-stories-33M",
# })

In [ ]:
# Set parameters for dataset size
cfg.input_len = 1
cfg.output_len = 25
cfg.num_targets = 100
cfg.max_chunk_size = 20
cfg.max_epochs = 100 * 1000
cfg.max_batch_size = {
    "tiny-stories-33M" : [300,300,280,270,260,250,250,240,240,230],
}[cfg.model_name][cfg.input_len-1]
cfg.max_batch_size = cfg.max_batch_size - 15

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

batch_size = 1000
for batch in range(0, cfg.num_targets, batch_size):
    input_tokens = true_tokens[batch:batch+batch_size].to(device)
    output_tokens = model.generate(
        input_tokens,
        # min_new_tokens=cfg.output_len,
        max_new_tokens=cfg.output_len,
        # do_sample=True,
        do_sample=False,
        stop_at_eos=False,
        verbose=False,
        return_type="tokens",)[:,cfg.input_len:]
    if batch == 0:
        all_output_tokens = output_tokens
    else:
        all_output_tokens = torch.cat((all_output_tokens, output_tokens), dim=0)
with open(DATA_PATH / f"true_tokens_{cfg.num_targets}_{cfg.input_len}_{cfg.output_len}_greedy.pkl", 'wb') as file:
    pickle.dump(all_output_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Generate the initialisations used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
initial_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(initial_tokens, file)

results, elapsed_time = pez_text_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)


INITIALISING STATE
(0/100)(100/100)100, (0/100)(100/100)200, (1/100)(100/100)300, (2/100)(100/100)400, (2/100)(100/100)500, (3/100)(100/100)600, (5/100)(100/100)700, (6/100)(100/100)800, (6/100)(100/100)900, (6/100)(100/100)1000, 


100%|██████████| 1/1 [00:00<00:00, 2557.50it/s]

input_len 1 	percent_zero_loss 6.0 	percent_exact_inversion 3.0 	elapsed_time 110.836


## PEZ Logits Inversion

In [6]:
from inversion_optimisation.algorithms.pez import pez_search
from inversion_optimisation.inv_configs import PEZ_LOGITS_CFG

cfg = PEZ_LOGITS_CFG
# cfg = DotDict({
#     "learn_rate" : 1.7,
#     "con_distance" : {
#         0: "cos sim",
#         1: "euc dist",
#     }[1],
#     "target_strategy" : {
#         0: "random",
#         1: "tinystories",
#         2: "reddit",
#         3: "wikipedia",
#         4: "privacy",
#     }[0],
#     "init_strategy" : {
#         0: "loaded",
#         1: "normal",
#         2: "zeros",
#     }[0],
#     "save_folder": "PEZ_TinyStories33M",
#     "model_name": "tiny-stories-33M",
# })

In [ ]:
# Set parameters for dataset size
cfg.input_len = 2
cfg.num_targets = 1000
cfg.max_epochs = 100 * 1000
# cfg.max_chunk_size = [30,20,15,10,8,6,6,5,5,4][cfg.input_len-1] # L4 batch with 100 batch
cfg.max_chunk_size = [20,15,10,7,6,5,4,3,2,2][cfg.input_len-1] # L4 batch with 1000 batch
cfg.max_batch_size = [1000,1000,1000,1000,1000,1000,1000,1000,1000,1000][cfg.input_len-1] # L4 batch

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Generate the initialisations used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
initial_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(initial_tokens, file)

results, elapsed_time = pez_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)

100%|██████████| 100/100 [00:00<00:00, 137.67it/s]


INITIALISING STATE
(1/100)(100/100)100, (1/100)(100/100)200, (2/100)(100/100)300, (2/100)(100/100)400, (3/100)(100/100)500, (3/100)(100/100)600, (4/100)(100/100)700, (5/100)(100/100)800, (5/100)(100/100)900, (6/100)(100/100)1000, 


100%|██████████| 3/3 [00:00<00:00, 7202.58it/s]

input_len 3 	percent_zero_loss 6.0 	percent_exact_inversion 6.0 	elapsed_time 96.652


## Constrained PIA Text-Only Inversion

In [ ]:
from inversion_optimisation.algorithms.constrained_pia import pia_text_search
from inversion_optimisation.inv_configs import PIA_TEXT_CFG

cfg = PIA_TEXT_CFG
# cfg = DotDict({
#     "learn_rate" : 3.5,
#     "top_k" : 50256,
#     "nn_weight" : 0.001,
#     "target_sample" : {
#         0: "random",
#         1: "greedy"
#     }[1],
#     "target_strategy" : {
#         0: "random",
#         1: "tinystories",
#         2: "reddit",
#         3: "wikipedia"
#     }[0],
#     "init_strategy" : {
#         0: "loaded",
#         1: "normal",
#         2: "zeros",
#     }[1],
#     "save_folder": "PIA_TinyStories33M_text",
#     "model_name": "tiny-stories-33M",
# })

In [ ]:
# Set parameters for dataset size
cfg.input_len = 1
cfg.output_len = 25
cfg.num_targets = 1000
cfg.max_chunk_size = 20
cfg.max_epochs = 100 * 1000
cfg.max_batch_size = {
    "tiny-stories-33M" : [300,300,280,270,260,250,250,240,240,230],
}[cfg.model_name][cfg.input_len-1]
cfg.max_batch_size = cfg.max_batch_size - 15

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

batch_size = 1000
for batch in range(0, cfg.num_targets, batch_size):
    input_tokens = true_tokens[batch:batch+batch_size].to(device)
    output_tokens = model.generate(
        input_tokens,
        # min_new_tokens=cfg.output_len,
        max_new_tokens=cfg.output_len,
        # do_sample=True,
        do_sample=False,
        stop_at_eos=False,
        verbose=False,
        return_type="tokens",)[:,cfg.input_len:]
    if batch == 0:
        all_output_tokens = output_tokens
    else:
        all_output_tokens = torch.cat((all_output_tokens, output_tokens), dim=0)
with open(DATA_PATH / f"true_tokens_{cfg.num_targets}_{cfg.input_len}_{cfg.output_len}_greedy.pkl", 'wb') as file:
    pickle.dump(all_output_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Generate the initialisations used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
initial_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(initial_tokens, file)

results, elapsed_time = pia_text_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)


## Constrained PIA Logits Inversion

In [ ]:
from inversion_optimisation.algorithms.constrained_pia import pia_search
from inversion_optimisation.inv_configs import PIA_LOGITS_CFG

cfg = PIA_LOGITS_CFG
# cfg = DotDict({
#     "learn_rate" : 3.5,
#     "top_k" : 50256,
#     "nn_weight" : 0.001,
#     "target_strategy" : {
#         0: "random",
#         1: "tinystories",
#         2: "reddit",
#         3: "wikipedia"
#     }[0],
#     "init_strategy" : {
#         0: "loaded",
#         1: "normal",
#         2: "zeros",
#     }[1],
#     "save_folder": "PIA_TinyStories33M",
#     "model_name": "tiny-stories-33M",
# })

In [ ]:
# Set parameters for dataset size
cfg.input_len = 2
cfg.num_targets = 1000
cfg.max_epochs = 100 * 1000
# cfg.max_chunk_size = [30,20,15,10,8,6,6,5,5,4][cfg.input_len-1] # L4 batch with 100 batch
cfg.max_chunk_size = [20,15,10,7,6,5,4,3,2,2][cfg.input_len-1] # L4 batch with 1000 batch
cfg.max_batch_size = [1000,1000,1000,1000,1000,1000,1000,1000,1000,1000][cfg.input_len-1] # L4 batch

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Generate the initialisations used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
initial_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(initial_tokens, file)

results, elapsed_time = pia_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)

## ARCA Text-Only Inversion

In [ ]:
from inversion_optimisation.algorithms.arca import arca_text_search
from inversion_optimisation.inv_configs import ARCA_TEXT_CFG

cfg = ARCA_TEXT_CFG
# cfg = DotDict({
#     "top_k" : 64,
#     "num_grad_samples" : 10,
#     "token_choice" : {
#         0: "uniform",
#         1: "weighted",
#     }[1],
#     "num_mutations" : 1,
#     "target_sample" : {
#         0: "random",
#         1: "greedy"
#     }[1],
#     "target_strategy" : {
#         0: "random",
#         1: "tinystories",
#         2: "reddit",
#         3: "wikipedia"
#     }[0],
#     "init_strategy" : {
#         0: "loaded",
#         1: "zeros",
#     }[0],
#     "save_folder": "ARCA_TinyStories33M_text",
#     "model_name": "tiny-stories-33M",
# })

In [ ]:
# Set parameters for dataset size
cfg.output_len = 25
cfg.input_len = 1
cfg.num_targets = 1000
cfg.num_candidates = (32*22)
cfg.adjusted_max_epochs = (32*22)
cfg.max_batch_size = {
    "tiny-stories-33M" : [300,300,280,270,260,250,250,240,240,230],
}[cfg.model_name][cfg.input_len-1]

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets and initialisations used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

batch_size = 1000
for batch in range(0, cfg.num_targets, batch_size):
    input_tokens = true_tokens[batch:batch+batch_size].to(device)
    output_tokens = model.generate(
        input_tokens,
        max_new_tokens=cfg.output_len,
        do_sample=False,
        stop_at_eos=False,
        verbose=False,
        return_type="tokens",)[:,cfg.input_len:]
    if batch == 0:
        all_output_tokens = output_tokens
    else:
        all_output_tokens = torch.cat((all_output_tokens, output_tokens), dim=0)
with open(DATA_PATH / f"true_tokens_{cfg.num_targets}_{cfg.input_len}_{cfg.output_len}_greedy.pkl", 'wb') as file:
    pickle.dump(all_output_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f"initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl", 'wb') as file:
    pickle.dump(true_tokens, file)

results, elapsed_time = arca_text_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.adjusted_max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.adjusted_max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.adjusted_max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)

INITIALISING STATE
(33/100)(100/100)50, 


100%|██████████| 1/1 [00:00<00:00, 2190.24it/s]

input_len 1 	percent_zero_loss 34.0 	percent_exact_inversion 14.0 	elapsed_time 53.387


## ARCA Logits Inversion

In [ ]:
from inversion_optimisation.algorithms.arca import arca_search
from inversion_optimisation.inv_configs import ARCA_LOGITS_CFG

cfg = ARCA_LOGITS_CFG
# cfg = DotDict({
#     "top_k" : 128,
#     "num_grad_samples" : 10,
#     "token_choice" : {
#         0: "uniform",
#         1: "weighted",
#     }[0],
#     "num_mutations" : 1,
#     "target_strategy" : {
#         0: "random",
#         1: "tinystories",
#         2: "reddit",
#         3: "wikipedia",
#         4: "privacy",
#     }[0],
#     "init_strategy" : {
#         0: "loaded",
#         1: "zeros",
#     }[0],
#     "save_folder": "ARCA_TinyStories33M",
#     "model_name": "tiny-stories-33M",
# })

In [ ]:
# Set parameters for dataset size
cfg.input_len = 2
cfg.num_targets = 1000
cfg.num_candidates = (32*22)
cfg.adjusted_max_epochs = (32*22)
cfg.max_batch_size = {
    "tiny-stories-33M" : [1000,1000,1000,1000,1000,1000,1000,900,750,650,None, None, None, None, 500, 500, 475, 450, 420, 400, 375, 350, 325, 300],
}[cfg.model_name][cfg.input_len-1]

# Set random seeds
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)

# Generate the targets and initialisations used for all experiments
tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f'true_tokens_{cfg.num_targets}_{cfg.input_len}.pkl', 'wb') as file:
    pickle.dump(true_tokens, file)

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

tokens_list = []
for _ in tqdm(range(cfg.num_targets)):
    tokens = torch.randint(0, len(model.tokenizer.vocab), (1, cfg.input_len)).to(device)
    tokens_list.append(tokens)
true_tokens = torch.cat(tokens_list, dim=0).to(device)
with open(DATA_PATH / f"initial_tokens_{cfg.num_targets}_{cfg.input_len}.pkl", 'wb') as file:
    pickle.dump(true_tokens, file)

results, elapsed_time = arca_search(cfg, model, device)
print()
stats = get_paper_summary_stats_new(results, epochs=cfg.adjusted_max_epochs)
stats["elapsed_time"] = elapsed_time
stats["experiment_params"] = cfg
print("input_len", cfg.input_len, "\tpercent_zero_loss", stats["percent_zero_loss"], "\tpercent_exact_inversion", stats["percent_exact_inversion"], "\telapsed_time", stats["elapsed_time"])

# with open(DATA_PATH / f'{cfg.save_folder}/stats_{cfg.input_len}_{cfg.num_targets}_{cfg.adjusted_max_epochs}.json', 'w') as f:
#     json.dump(stats, f)
# with open(DATA_PATH / f'{cfg.save_folder}/results_{cfg.input_len}_{cfg.num_targets}_{cfg.adjusted_max_epochs}.pkl', 'wb') as f:
#     pickle.dump(results, f)